In [1]:
import xarray as xr
import numpy as np
import json

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

# Direct access to variables
u10 = ds["U10"].isel(Time=0)     # shape: (y, x)
v10 = ds["V10"].isel(Time=0)
lat = ds["XLAT"].isel(Time=0)
lon = ds["XLONG"].isel(Time=0)

# Prepare leaflet-velocity JSON format
ny, nx = u10.shape

data = {
    "header": {
        "parameterUnit": "m/s",
        "parameterNumber": 2,
        "parameterNumberName": "eastward_wind",
        "parameterCategory": 2,
        "nx": nx,
        "ny": ny,
        "lo1": float(lon[0, 0]),
        "la1": float(lat[0, 0]),
        "lo2": float(lon[-1, -1]),
        "la2": float(lat[-1, -1]),
        "dx": float((lon[0, -1] - lon[0, 0]) / (nx - 1)),
        "dy": float((lat[0, 0] - lat[-1, 0]) / (ny - 1)),
        "refTime": "2024-02-19 00:00:00",
        "forecastTime": 0
    },
    "uComponent": u10.values.flatten().tolist(),
    "vComponent": v10.values.flatten().tolist()
}

# Save to JSON
with open("wind_velocity.json", "w") as f:
    json.dump(data, f)

print("✅ wind_velocity.json created without wrf-python.")

✅ wind_velocity.json created without wrf-python.


In [6]:
import xarray as xr
import numpy as np
import json

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

# Extract U10 and V10 (10 m winds)
u10 = ds["U10"].isel(Time=0)
v10 = ds["V10"].isel(Time=0)
lat = ds["XLAT"].isel(Time=0)
lon = ds["XLONG"].isel(Time=0)

# Convert from m/s to knots
u10_knots = u10.values * 1.94384
v10_knots = v10.values * 1.94384

ny, nx = u10.shape

# Shared header metadata
base_header = {
    "parameterUnit": "KT",
    "parameterCategory": 2,
    "nx": nx,
    "ny": ny,
    "lo1": float(lon[0, 0]),
    "la1": float(lat[0, 0]),
    "lo2": float(lon[-1, -1]),
    "la2": float(lat[-1, -1]),
    "dx": float((lon[0, -1] - lon[0, 0]) / (nx - 1)),
    "dy": float((lat[0, 0] - lat[-1, 0]) / (ny - 1)),
    "refTime": "2024-05-01 00:00:00",
    "forecastTime": 0
}

# Create two objects: eastward and northward winds
eastward = {
    "header": {**base_header, "parameterNumber": 2, "parameterNumberName": "eastward_wind"},
    "data": u10_knots.flatten().tolist()
}

northward = {
    "header": {**base_header, "parameterNumber": 3, "parameterNumberName": "northward_wind"},
    "data": v10_knots.flatten().tolist()
}

# Combine into one array (expected by Leaflet Velocity)
combined = [eastward, northward]

# Write to file
with open("wind_velocity.json", "w") as f:
    json.dump(combined, f)

print("✅ wind_velocity.json created with both eastward and northward wind components.")


✅ wind_velocity.json created with both eastward and northward wind components.
